Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

## Quick Start Notebook

This notebook shows how to train a Llama 2 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA.

### Step 0: Install pre-requirements and convert checkpoint

The example uses the Hugging Face trainer and model which means that the checkpoint has to be converted from its original format into the dedicated Hugging Face format.
The conversion can be achieved by running the `convert_llama_weights_to_hf.py` script provided with the transformer package.
Given that the original checkpoint resides under `models/7B` we can install all requirements and convert the checkpoint with:

In [1]:
# %%bash
# pip install llama-recipes transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# python ${TRANSFORM} --input_dir models --model_size 7B --output_dir models_hf_with_rag/7B

In [1]:
#import transformers
#print('/'.join(transformers.__file__.split('/')[:-1]) + '/models/llama/convert_llama_weights_to_hf.py')

In [2]:
#%%bash
#python '/jet/home/apatula/.local/lib/python3.10/site-packages/transformers/models/llama/convert_llama_weights_to_hf.py' --input_dir llama-2-7b --model_size 7B --output_dir models_hf/7B

### Step 0: RAG & DataPreparation 

In [3]:
# !pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

In [4]:
%pip install --upgrade --quiet  "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.


In [5]:
# !pip install opencv-python

In [6]:
# !pip install sentence-transformers

In [7]:
### Documents Loader

In [41]:
import torch
torch.cuda.empty_cache() 
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Mar  3 22:53:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM3-32GB           On  | 00000000:BC:00.0 Off |                    0 |
| N/A   39C    P0              71W / 350W |  10503MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [42]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, TextLoader

DOCS_DIR = './docs'
loader = DirectoryLoader(DOCS_DIR, glob='**/*.txt', show_progress=True, loader_cls=TextLoader)
docs = loader.load()
len(docs)

100%|██████████| 29/29 [00:00<00:00, 59.65it/s]


29

In [43]:
### Split Files into chunks

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
len(splits)

1363

In [5]:
# Embed
from chromadb.utils import embedding_functions
from langchain_community.embeddings.huggingface import (HuggingFaceEmbeddings)

embeddings_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)

vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeddings)

retriever = vectorstore.as_retriever()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:836: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
import pandas as pd    
qa = pd.read_json(path_or_buf='QAndA_JSONL.txt', lines=True)
qa.head()

,input,output
0,"How do China's AI regulations, particularly o...",China's AI regulations prioritize information ...
1,Why should international discourse take China'...,China's AI regulations reshape global AI deplo...
2,What are the structural similarities found in ...,China's AI regulations share three structural ...
3,How does China formulate AI governance regulat...,China formulates AI governance regulations thr...
4,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...


In [8]:
import json
import pandas as pd

#path to the json file containing all the input output pairs of all the 5 teams
train_file_path = 'merged_file_train.json'
val_file_path = 'merged_file_val.json'
test_file_path = 'merged_file_test.json'

In [9]:
# Read and handle UTF-8 BOM if present
with open(train_file_path, 'r', encoding='utf-8-sig') as file:
    json_data = json.load(file)  # Load the file content as JSON
    
# Convert the JSON data into a pandas DataFrame
train_dataframe = pd.DataFrame(json_data)

In [10]:
# Read and handle UTF-8 BOM if present
with open(val_file_path, 'r', encoding='utf-8-sig') as file:
    json_data = json.load(file)  # Load the file content as JSON
    
# Convert the JSON data into a pandas DataFrame
val_dataframe = pd.DataFrame(json_data)

In [11]:
# Read and handle UTF-8 BOM if present
with open(test_file_path, 'r', encoding='utf-8-sig') as file:
    json_data = json.load(file)  # Load the file content as JSON
    
# Convert the JSON data into a pandas DataFrame
test_dataframe = pd.DataFrame(json_data)

In [12]:
#Retrieve 4 Relevant documents chunks - Feel free to change
#results = vectorstore.similarity_search(query, k=4)
train_dataframe['context'] = train_dataframe['input'].apply(lambda x: ' '.join([result.page_content for result in vectorstore.similarity_search(x, k=4)]))
train_dataframe.head()

,input,output,context
0,"How do China's AI regulations, particularly o...",China's AI regulations prioritize information ...,China’s three most concrete and impactful regu...
1,Why should international discourse take China'...,China's AI regulations reshape global AI deplo...,But international discourse on Chinese AI gove...
2,What are the structural similarities found in ...,China's AI regulations share three structural ...,2 AI legislation in China And despite China’s ...
3,Why was the Artificial Intelligence Act intro...,"To ensure safe, rights-respecting AI while pro...",AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...
4,What applications are banned under the Act int...,Banned applications include sensitive biometri...,The EU’s AI Act intends to be the “world’s fir...


In [13]:
processed_train_dataframe = train_dataframe
processed_train_dataframe.to_json('train_qa_with_context.jsonl', orient='records', lines=True)

In [14]:
#Retrieve 4 Relevant documents chunks - Feel free to change
#results = vectorstore.similarity_search(query, k=4)
val_dataframe['context'] = val_dataframe['input'].apply(lambda x: ' '.join([result.page_content for result in vectorstore.similarity_search(x, k=4)]))
val_dataframe.head()

,input,output,context
0,How does China formulate AI governance regulat...,China formulates AI governance regulations thr...,How China Sets AI Governance Policy\nThis pape...
1,What obligations apply to high-risk AI systems...,High-risk AI systems must undergo fundamental ...,Although existing legislation provides some pr...
2,What obligations apply to high-risk AI systems...,"Conformity assessment with risk, quality, trac...",Although existing legislation provides some pr...
3,Does the UK AI regulatory framework propose ne...,"No, the government avoids new legal requiremen...",UK Government Publishes AI Regulatory Framewor...
4,What are some of the current regulations in th...,"Currently, the UK has laws in effect that gove...",largely regulates AI via existing laws enforce...


In [15]:
processed_val_dataframe = val_dataframe
processed_val_dataframe.to_json('val_qa_with_context.jsonl', orient='records', lines=True)

In [16]:
#Retrieve 4 Relevant documents chunks - Feel free to change
#results = vectorstore.similarity_search(query, k=4)
test_dataframe['context'] = test_dataframe['input'].apply(lambda x: ' '.join([result.page_content for result in vectorstore.similarity_search(x, k=4)]))
test_dataframe.head()

,input,output,context
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt..."
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al...."


In [17]:
processed_test_dataframe = test_dataframe
processed_test_dataframe.to_json('test_qa_with_context.jsonl', orient='records', lines=True)

### Step 1: Load the model

Point base_model to model weight folder

In [18]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [7]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

logging.set_verbosity(logging.ERROR)
base_model = "./models_hf_with_rag/7B"
finetuned_model = './finetuned_with_rag/policy-llama2-7b'

train_dataset = Dataset(pa.Table.from_pandas(pd.read_json(path_or_buf='train_qa_with_context.jsonl', lines=True)))
val_dataset = Dataset(pa.Table.from_pandas(pd.read_json(path_or_buf='val_qa_with_context.jsonl', lines=True)))

In [9]:
train_dataset[0]

{'input': " How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?",
 'output': 'China\'s AI regulations prioritize information control through measures like barring excessive price discrimination in recommendation algorithms, requiring labels on synthetically generated content in deep synthesis, and demanding "true and accurate" data and outputs in generative AI.',
 'context': 'China’s three most concrete and impactful regulations on algorithms and AI are its 2021\nregulation on recommendation algorithms, the 2022 rules for deep synthesis (synthetically generated content), and the 2023 draft rules on generative AI. Information control is a central goal of all three measures, but they also contain many other notable provisions. The rules for recommendation algorithms bar excessive price discrimination and protect the rights of workers subject to algorithmic scheduling. \nThe deep synthesis regulation re

In [12]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [44]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    #device_map={"": 0, "":1}
    device_map='auto'
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Step 2: Load LLaMA tokenizer


In [45]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [25]:
eval_prompt = """
Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:

China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control by requiring companies to obtain approval from the government before using these technologies. This is done to ensure that the information being used is accurate and not misleading, and to prevent the spread of false or harmful information.

For example, the Chinese government has implemented regulations that require companies to obtain approval before using recommendation algorithms to suggest products


In [26]:
#With Context
eval_prompt = """
Answer the following question with the given context with succinct summary:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
Context: China\u2019s three most concrete and impactful regulations on algorithms and AI are its 2021\nregulation on recommendation algorithms, the 2022 rules for deep synthesis (synthetically generated content), and the 2023 draft rules on generative AI. Information control is a central goal of all three measures, but they also contain many other notable provisions. The rules for recommendation algorithms bar excessive price discrimination and protect the rights of workers subject to algorithmic scheduling. \nThe deep synthesis regulation requires conspicuous labels be placed on synthetically generated content. And the draft generative AI regulation requires both the training data and model outputs to be \u201ctrue and accurate,\u201d Summary: China is in the midst of rolling out some of the world\u2019s earliest and most detailed regulations governing artificial intelligence (AI). These include measures governing recommendation algorithms\u2014the most omnipresent form of AI deployed on the internet\u2014as well as new rules for synthetically generated images and chatbots in the mold of ChatGPT. \nChina\u2019s emerging AI governance framework will reshape how the technology is built and deployed within China and internationally, impacting both Chinese technology exports and global AI research networks.\nBut in the West, China\u2019s regulations are often dismissed as irrelevant or seen purely through the lens of a geopolitical competition to write the rules for AI. These extremely demanding requirements for generative AI systems have kicked off a particularly active public debate on the draft regulation. At the time of writing, Chinese scholars, companies, and policymakers are actively discussing how to maintain effective content controls without squashing China\u2019s nascent generative AI industry. The third paper in this series will dive deep into how this policy debate is playing out in public workshops, academic writing, and corporate lobbying.\nCountries and cultures may differ on the specific content of AI regulations, but they can learn from the content-agnostic structure of the regulations themselves. The above Chinese regulations share three structural similarities: the choice of algorithms as a point of entry; the building of regulatory tools and bureaucratic know-how; and the vertical and iterative approach that is laying the groundwork for a capstone AI law. Three regulations require the deepest analysis: recommendation algorithms, \u201cdeep synthesis,\u201d and generative AI. These interconnected documents contain the most targeted and impactful regulations to date, creating concrete requirements for how algorithms and AI are built and deployed in China. Below is a brief overview of each regulation. The remainder of this paper and subsequent papers will expand on the intellectual roots and key bureaucratic actors behind these regulations.\nProvisions on the Management of Algorithmic Recommendations in Internet Information Services
---
Answer:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=200)[0], skip_special_tokens=True))


Answer the following question with the given context with succinct summary:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
Context: China’s three most concrete and impactful regulations on algorithms and AI are its 2021
regulation on recommendation algorithms, the 2022 rules for deep synthesis (synthetically generated content), and the 2023 draft rules on generative AI. Information control is a central goal of all three measures, but they also contain many other notable provisions. The rules for recommendation algorithms bar excessive price discrimination and protect the rights of workers subject to algorithmic scheduling. 
The deep synthesis regulation requires conspicuous labels be placed on synthetically generated content. And the draft generative AI regulation requires both the training data and model outputs to be “true and accurate,” Summary: China is in the midst of rolling out

### Step 3: Load LoRA configuration

In [27]:
peft_params = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules = ["q_proj", "v_proj"],
    task_type="CAUSAL_LM",
)

### Step 4: Set training parameters

In [28]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

### Step 5: Set supervised fine-tuning parameters

In [29]:
train_dataset[0]

{'input': " How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?",
 'output': 'China\'s AI regulations prioritize information control through measures like barring excessive price discrimination in recommendation algorithms, requiring labels on synthetically generated content in deep synthesis, and demanding "true and accurate" data and outputs in generative AI.',
 'context': 'China’s three most concrete and impactful regulations on algorithms and AI are its 2021\nregulation on recommendation algorithms, the 2022 rules for deep synthesis (synthetically generated content), and the 2023 draft rules on generative AI. Information control is a central goal of all three measures, but they also contain many other notable provisions. The rules for recommendation algorithms bar excessive price discrimination and protect the rights of workers subject to algorithmic scheduling. \nThe deep synthesis regulation re

In [30]:
val_dataset[0]

{'input': 'How does China formulate AI governance regulations, and what are the key layers in the policy formulation process?',
 'output': 'China formulates AI governance regulations through a four-layered policy funnel involving real-world roots, Xi Jinping and CCP ideology, the "world of ideas," and party and state bureaucracies, with regulations often pinballing through these layers in a non-linear fashion.',
 'context': 'How China Sets AI Governance Policy\nThis paper presents a four-layered policy funnel through which China formulates and pro-mulgates AI governance regulations. Those four layers are real-world roots; Xi Jinping and CCP ideology; the “world of ideas”; and the party and state bureaucracies. These layers are porous, and regulations do not proceed through them in a purely linear fashion.\n\nInstead, they often pinball forward and backward through these layers, getting shaped and reshaped by academics, bureaucrats, public opinion, and CCP ideology. The order and relati

In [31]:
def formatting_prompts_func(example) -> list:
    output_texts = []
    for i in range(len(example['input'])):
        text = f"### Question: {example['input'][i]}\n ### Answer: {example['output'][i]}\n ### Context: {example['context'][i]}"
        output_texts.append(text)
    return output_texts

In [32]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset = val_dataset,
    peft_config=peft_params,
    #dataset_text_field="input",
    formatting_func=formatting_prompts_func,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/262 [00:00<?, ? examples/s]

Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [33]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Mar  3 18:01:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   33C    P0              68W / 300W |   7237MiB / 32768MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Step 6: Train Model


In [34]:
torch.cuda.empty_cache() 
trainer.train()

{'loss': 2.0025, 'grad_norm': 0.664797842502594, 'learning_rate': 0.0002, 'epoch': 0.38}


/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf_with_rag/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.878, 'grad_norm': 0.7222964763641357, 'learning_rate': 0.0002, 'epoch': 0.76}


/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf_with_rag/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 1.6979, 'grad_norm': 1.0306320190429688, 'learning_rate': 0.0002, 'epoch': 1.14}


...

/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf_with_rag/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'loss': 0.3348, 'grad_norm': 2.268657922744751, 'learning_rate': 0.0002, 'epoch': 9.85}


/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf_with_rag/7B - will assume that the vocabulary was not modified.
  warnings.warn(


{'train_runtime': 971.1715, 'train_samples_per_second': 2.698, 'train_steps_per_second': 0.68, 'train_loss': 0.9685058658773249, 'epoch': 10.0}


TrainOutput(global_step=660, training_loss=0.9685058658773249, metrics={'train_runtime': 971.1715, 'train_samples_per_second': 2.698, 'train_steps_per_second': 0.68, 'train_loss': 0.9685058658773249, 'epoch': 10.0})

In [35]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Sun Mar  3 18:19:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   51C    P0              80W / 300W |  32483MiB / 32768MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [36]:
# Save trained model
trainer.model.save_pretrained(finetuned_model)
trainer.tokenizer.save_pretrained(finetuned_model)

/jet/home/achoudh2/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:148: UserWarning: Could not find a config file in ./models_hf_with_rag/7B - will assume that the vocabulary was not modified.
  warnings.warn(


('./finetuned_with_rag/policy-llama2-7b/tokenizer_config.json',
 './finetuned_with_rag/policy-llama2-7b/special_tokens_map.json',
 './finetuned_with_rag/policy-llama2-7b/tokenizer.model',
 './finetuned_with_rag/policy-llama2-7b/added_tokens.json',
 './finetuned_with_rag/policy-llama2-7b/tokenizer.json')

In [46]:
# load fine-tuned tokenizer
tokenizer2 = AutoTokenizer.from_pretrained(finetuned_model, trust_remote_code=True)
tokenizer2.pad_token = tokenizer2.eos_token
tokenizer2.padding_side = "right"

In [47]:
# load fine-tuned model
model2 = AutoModelForCausalLM.from_pretrained(
    finetuned_model,
    quantization_config=quant_config,
    #device_map={"": 0, "":1}
    device_map='auto'
)
model2.config.use_cache = False
model2.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
eval_prompt = """
Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
"""

model_input2 = tokenizer2(eval_prompt, return_tensors="pt").to("cuda")

model2.eval()
with torch.no_grad():
    print(tokenizer2.decode(model2.generate(**model_input2, max_new_tokens=100)[0], skip_special_tokens=True))


Answer the following question:
Question: How do China's AI regulations, particularly on recommendation algorithms, deep synthesis, and generative AI, focus on information control?
---
Answer:
China's AI regulations emphasize information control through measures like banning recommendation algorithms that spread extreme or superstitious content, banning deep synthesis that creates fake images of people, and requiring generative AI to include a disclaimer stating it is artificially generated.

In 2021, China issued its first official AI regulations, the draft AI ethics principles. These principles emphasize information control, with specific measures including banning


In [15]:
%load_ext tensorboard
from tensorboard import notebook
log_dir = "results/runs"
notebook.start("--logdir {} --port 4000".format(log_dir))

In [38]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    #device_map={"": 0, "":1}
    device_map='auto'
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [39]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [16]:
import pandas as pd

def zero_shot_prompts(data):
    prompt_list = data['input'].tolist()
    return prompt_list

In [17]:
# remove input prompt from output
def remove_redundant(input_text, output_text):
    if output_text.startswith(input_text):
        return output_text[len(input_text):].strip()
    return output_text

In [18]:
# enter prompts

def test_prompts(tokenizer, model, prompt_list):
    outputs = []

    for prompt in prompt_list:

        model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

        model.eval()
        with torch.no_grad():
            output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
            #print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
            #print("=========================================================================================")
            #print(output)
            outputs.append(output)
    return outputs

In [21]:
# Additional article for types of prompt engineering: https://medium.com/@amiraryani/8-types-of-prompt-engineering-5322fff77bdf
#Zero-shot Prompting

file = 'test_qa_with_context.jsonl'
data = pd.read_json(file, lines=True)
data.rename(columns={'output': 'true_output'}, inplace=True)
prompt_list = zero_shot_prompts(data)
#print(prompt_list)

# prompt_list = ["List the most pressing topics regarding regulation of AI.",
#                "Who owns material generated by a company’s large language model?",
#                "Describe how China and the United States are approaching making new legislation to regulate Generative AI?"
#                ]

outputs = test_prompts(tokenizer2, model2, prompt_list)

data['output_fine_tuned_zero'] = outputs
data['output_fine_tuned_zero'] = data.apply(lambda row: remove_redundant(row['input'], row['output_fine_tuned_zero']), axis=1)
data.to_csv('prompt_outputs.csv')
data.head()

,input,true_output,context,output_fine_tuned_zero
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt...",China is the only country that has implemented...
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...,The Europe Artificial Intelligence Act (AAI) r...
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...,The European Commission published a Staff Work...
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...,The UK’s fourth evaluation report on AI regula...
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al....",How does the UK government define Generative A...


In [40]:
# base model outputs
outputs = test_prompts(tokenizer, model, prompt_list)

data['output_base_zero'] = outputs
data['output_base_zero'] = data.apply(lambda row: remove_redundant(row['input'], row['output_base_zero']), axis=1)
data.to_csv('prompt_outputs2.csv')
data.head()

,input,true_output,context,output_fine_tuned_zero,output_fine_tuned_one,output_fine_tuned_few,output_fine_tuned_neg,output_fine_tuned_cot,output_base_zero
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt...",China is the only country that has implemented...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,"sierp 23, 2022\nChina’s AI governance is a com..."
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...,The Europe Artificial Intelligence Act (AAI) r...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,"sierp 23, 2022\nThe European Commission has pu..."
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...,The European Commission published a Staff Work...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,sierpniu 2021\nThe European Commission has pub...
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...,The UK’s fourth evaluation report on AI regula...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,sierp 2018\nThe UK regulatory framework for fi...
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al....",How does the UK government define Generative A...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,"sierp 23, 2022\nGenerative AI is a type of art..."


In [22]:
def one_shot_prompting(prompt_list):
    prompt_list2 = []
    for prompt in prompt_list:
        
        prompt2 = '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
                   and moral issues regarding the development, testing, evaluation, and use of Generative AI:
                   Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
                   =>
                   The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
                   and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
                   exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
                   private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
                   Prompt: ''' + prompt + ''' => '''
        prompt_list2.append(prompt2)
    return prompt_list2

In [23]:
one_shot_list = one_shot_prompting(prompt_list)
outputs = test_prompts(tokenizer2, model2, one_shot_list)

data['output_fine_tuned_one'] = outputs
data['output_fine_tuned_one'] = data.apply(lambda row: remove_redundant(row['input'], row['output_fine_tuned_one']), axis=1)
data.to_csv('prompt_outputs.csv')
data.head()

,input,true_output,context,output_fine_tuned_zero,output_fine_tuned_one
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt...",China is the only country that has implemented...,Task: You are a chat bot called AI PolicyChat....
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...,The Europe Artificial Intelligence Act (AAI) r...,Task: You are a chat bot called AI PolicyChat....
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...,The European Commission published a Staff Work...,Task: You are a chat bot called AI PolicyChat....
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...,The UK’s fourth evaluation report on AI regula...,Task: You are a chat bot called AI PolicyChat....
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al....",How does the UK government define Generative A...,Task: You are a chat bot called AI PolicyChat....


In [24]:
def few_shot_prompting(prompt_list):
    prompt_list3 = []
    for prompt in prompt_list:
        
        prompt3 = '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
               the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
               Prompt: ''' + prompt + ''' =>'''
        prompt_list3.append(prompt3)
    return prompt_list3

In [25]:
few_shot_list = few_shot_prompting(prompt_list)
outputs = test_prompts(tokenizer2, model2, few_shot_list)

data['output_fine_tuned_few'] = outputs
data['output_fine_tuned_few'] = data.apply(lambda row: remove_redundant(row['input'], row['output_fine_tuned_few']), axis=1)
data.to_csv('prompt_outputs.csv')
data.head()

,input,true_output,context,output_fine_tuned_zero,output_fine_tuned_one,output_fine_tuned_few
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt...",China is the only country that has implemented...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...,The Europe Artificial Intelligence Act (AAI) r...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...,The European Commission published a Staff Work...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...,The UK’s fourth evaluation report on AI regula...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al....",How does the UK government define Generative A...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....


In [26]:
def negative_prompting(prompt_list):
    prompt_list4 = []
    for prompt in prompt_list:
        
        prompt4 = '''Task: You are a chat bot called AI PolicyChat. Do not provide any commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI. Ensure that your answers are
               the opposite of the correct answer. Prompt: ''' + prompt + ''' => '''
        prompt_list4.append(prompt4)
    return prompt_list4

In [27]:
negative_list = negative_prompting(prompt_list)
outputs = test_prompts(tokenizer2, model2, negative_list)

data['output_fine_tuned_neg'] = outputs
data['output_fine_tuned_neg'] = data.apply(lambda row: remove_redundant(row['input'], row['output_fine_tuned_neg']), axis=1)
data.to_csv('prompt_outputs.csv')
data.head()

,input,true_output,context,output_fine_tuned_zero,output_fine_tuned_one,output_fine_tuned_few,output_fine_tuned_neg
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt...",China is the only country that has implemented...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...,The Europe Artificial Intelligence Act (AAI) r...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...,The European Commission published a Staff Work...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...,The UK’s fourth evaluation report on AI regula...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al....",How does the UK government define Generative A...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....


In [28]:
def cot_prompting(prompt_list):
    prompt_list5 = []
    for prompt in prompt_list:
        
        prompt5 = '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example:
               Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                International discourse on Chinese AI governance often fails to take these regulations seriously, 
                to engage with either their content or the policymaking process. International commentary often falls into one of two 
                traps: dismissing China's regulations as irrelevant or using them as a political prop. Analysts and policymakers in 
                other countries often treat them as meaningless pieces of paper.
                China's AI regulations share three structural similarities: they focus on algorithms as a point of entry, build 
                regulatory tools and bureaucratic know-how, and adopt a vertical and iterative approach towards regulation.
                China's AI regulations reshape global AI deployment and offer insights into effective governance tools, 
                influencing various sectors worldwide.
                Prompt: ''' + prompt + ''' => '''
        
        prompt_list5.append(prompt5)
    return prompt_list5

In [29]:
cot_list = cot_prompting(prompt_list)
outputs = test_prompts(tokenizer2, model2, cot_list)

data['output_fine_tuned_cot'] = outputs
data['output_fine_tuned_cot'] = data.apply(lambda row: remove_redundant(row['input'], row['output_fine_tuned_cot']), axis=1)
data.to_csv('prompt_outputs.csv')
data.head()

,input,true_output,context,output_fine_tuned_zero,output_fine_tuned_one,output_fine_tuned_few,output_fine_tuned_neg,output_fine_tuned_cot
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt...",China is the only country that has implemented...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...,The Europe Artificial Intelligence Act (AAI) r...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...,The European Commission published a Staff Work...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...,The UK’s fourth evaluation report on AI regula...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al....",How does the UK government define Generative A...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....


In [35]:
pattern = r'Task:.*?Prompt:\s*'

data.columns
data2 = data.copy()
data2.head()

,input,true_output,context,output_fine_tuned_zero,output_fine_tuned_one,output_fine_tuned_few,output_fine_tuned_neg,output_fine_tuned_cot
0,What is the trajectory of Chinese AI governanc...,Chinese AI governance is heading towards draft...,"In this series of three papers, I will attempt...",China is the only country that has implemented...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
1,How does the Europe Artificial Intelligence Ac...,General-purpose AI systems must be transparent...,AN OVERVIEW OF NATIONAL AI STRATEGIES AND POLI...,The Europe Artificial Intelligence Act (AAI) r...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
2,How do providers of high-risk AI systems manag...,"Providers follow a step-by-step process, inclu...",step-by-step process for declaration of confor...,The European Commission published a Staff Work...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
3,What are the cross-sectoral principles of the ...,"The principles cover safety, transparency, fai...",Five cross-sectoral principles: When implement...,The UK’s fourth evaluation report on AI regula...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....
4,What are some of the jobs that may be affected...,Generative AI has the possibility to affect ma...,"for the Future of Work.\n33. Briggs, J. et al....",How does the UK government define Generative A...,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....,Task: You are a chat bot called AI PolicyChat....


In [ ]:
data2['output_fine_tuned_cot'] = data2['output_fine_tuned_cot'].str.replace(pattern, '', regex=True)
data2['output_fine_tuned_cot']

# Demonstration Prompts

## Base Model Few-Shot

In [55]:
eval_prompt = """
Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
               the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
               Prompt: How might AI affect competition between nations in the future? =>
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums f

## Fine-Tuned Model Few-Shot

In [56]:
eval_prompt = """

Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
               the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
               Prompt: How might AI affect competition between nations in the future? =>
"""

model_input2 = tokenizer2(eval_prompt, return_tensors="pt").to("cuda")

model2.eval()
with torch.no_grad():
    print(tokenizer2.decode(model2.generate(**model_input2, max_new_tokens=100)[0], skip_special_tokens=True))



Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
               =>
               The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
               and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
               exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
               private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
               Example: Which international forums 

## Base Model Chain-of-Thought

In [57]:
eval_prompt = """
Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example:
               Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                International discourse on Chinese AI governance often fails to take these regulations seriously, 
                to engage with either their content or the policymaking process. International commentary often falls into one of two 
                traps: dismissing China's regulations as irrelevant or using them as a political prop. Analysts and policymakers in 
                other countries often treat them as meaningless pieces of paper.
                China's AI regulations share three structural similarities: they focus on algorithms as a point of entry, build 
                regulatory tools and bureaucratic know-how, and adopt a vertical and iterative approach towards regulation.
                China's AI regulations reshape global AI deployment and offer insights into effective governance tools, 
                influencing various sectors worldwide.
                Prompt: How might AI affect competition between nations in the future? =>
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example:
               Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                International discourse on Chinese AI governance often fails to take these regulations seriously, 
                to engage with either their content or the policymaking process. International commentary often falls into one of two 
                traps: dismissing China's regulations as irre

## Fine-Tuned Model Chain-of-Thought

In [58]:
eval_prompt = """
Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example:
               Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                International discourse on Chinese AI governance often fails to take these regulations seriously, 
                to engage with either their content or the policymaking process. International commentary often falls into one of two 
                traps: dismissing China's regulations as irrelevant or using them as a political prop. Analysts and policymakers in 
                other countries often treat them as meaningless pieces of paper.
                China's AI regulations share three structural similarities: they focus on algorithms as a point of entry, build 
                regulatory tools and bureaucratic know-how, and adopt a vertical and iterative approach towards regulation.
                China's AI regulations reshape global AI deployment and offer insights into effective governance tools, 
                influencing various sectors worldwide.
                Prompt: How might AI affect competition between nations in the future? =>
"""

model_input2 = tokenizer2(eval_prompt, return_tensors="pt").to("cuda")

model2.eval()
with torch.no_grad():
    print(tokenizer2.decode(model2.generate(**model_input2, max_new_tokens=100)[0], skip_special_tokens=True))


Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
               and moral issues regarding the development, testing, evaluation, and use of Generative AI:
               Example:
               Question: Why should international discourse take China's AI regulations seriously? 
                Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
                Second, discuss China's most recent legislative changes regarding AI.
                Then, state why other nation's should pay close attention to China's new AI regulations. =>
                International discourse on Chinese AI governance often fails to take these regulations seriously, 
                to engage with either their content or the policymaking process. International commentary often falls into one of two 
                traps: dismissing China's regulations as irre

In [ ]:
#One-shot Prompting

# prompt_list2 = ['''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
#                    and moral issues regarding the development, testing, evaluation, and use of Generative AI:
#                    Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
#                    =>
#                    The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
#                    and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
#                    exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
#                    private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
#                    Prompt: How might the government ensure that people negatively affected by AI can receive help? =>''',
#                   '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, and moral issues 
#                   regarding the development, testing, evaluation, and use of Generative AI:
#                   Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
#                   the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
#                   Prompt: How do AI regulators attempt to enforce their regulations? =>''',
#                   '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
#                   and moral issues regarding the development, testing, evaluation, and use of Generative AI:
#                   Example: What are the possible biases that have been detected in healthcare ML produced by ML interactions with patients? =>
#                   In the ML-patient interaction case, it is possible to detect biases including: 
#                   Privilege bias, i.e. some models may be unavailable in settings where protected groups receive care or require technology/sensors 
#                   disproportionately available to the nonprotected class, and this also exacerbates existing inequalities between the ‘haves’ and 
#                   the ‘have-nots’ in terms of access to the digital healthcare ecosystem; in other words, those that generate enough data on themselves 
#                   to ensure accurately trained algorithms and those that do not. Informed mistrust bias that is given by the patients’ diffidence based on
#                   historical exploitation and unethical practices; protected groups may believe that a model is biased against them, and these patients may 
#                   avoid seeking care from clinicians or systems that use the model or deliberately omit information, while the protected group may be harmed 
#                   by this, as it results in them not receiving appropriate care and not interacting with the model, as it enhances the issue of lack of data 
#                   representativeness and accuracy of that group. Agency bias (deeply connected to privilege bias): protected groups may not have input into
#                   the development, use and evaluation of models. Thus, they may not have the resources, education or political influence to detect biases, 
#                   protest and force correction concerning the consideration or treatment of patients, especially those belonging to protected groups.
#                   Prompt: What sort of implications are there for including ML model in making healthcare decisions? =>''']

# for prompt in prompt_list2:
    
#     model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

#     model.eval()
#     with torch.no_grad():
#         output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print("=========================================================================================")
#         #print(output)
#     pair = {"input": prompt, "output": output}
#     input_output_pairs.append(pair)

In [ ]:
# #Few-shot Prompting
# prompt_list3 = ['''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
#                and moral issues regarding the development, testing, evaluation, and use of Generative AI:
#                Example: What is the purpose of the Executive Order on the safe, secure, and trustworthy development and use of artificial intelligence?
#                =>
#                The purpose of the Executive Order is to guide the development and use of artificial intelligence (AI) in a manner that is safe, secure, 
#                and trustworthy. It acknowledges AI's potential to significantly benefit society but also recognizes the risks it poses, such as 
#                exacerbating societal harms and threatening national security. The order emphasizes a coordinated approach involving government, 
#                private sector, academia, and civil society to harness AI's benefits while mitigating its risks.
#                Example: Which international forums focus on AI governance? =>  AI governance has been a focus of discussions in the G7, 
#                the U.S.-EU Trade and Technology Council, and the Global Partnership on AI (GPAI).
#                Prompt: How do AI regulators attempt to enforce their regulations? =>''',
#                '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
#                and moral issues regarding the development, testing, evaluation, and use of Generative AI:
#                Example: What are the possible biases that have been detected in healthcare ML produced by ML interactions with patients? =>
#                In the ML-patient interaction case, it is possible to detect biases including: 
#                Privilege bias, i.e. some models may be unavailable in settings where protected groups receive care or require technology/sensors 
#                disproportionately available to the nonprotected class, and this also exacerbates existing inequalities between the ‘haves’ and 
#                the ‘have-nots’ in terms of access to the digital healthcare ecosystem; in other words, those that generate enough data on themselves 
#                to ensure accurately trained algorithms and those that do not. Informed mistrust bias that is given by the patients’ diffidence based on
#                historical exploitation and unethical practices; protected groups may believe that a model is biased against them, and these patients may 
#                avoid seeking care from clinicians or systems that use the model or deliberately omit information, while the protected group may be harmed 
#                by this, as it results in them not receiving appropriate care and not interacting with the model, as it enhances the issue of lack of data 
#                representativeness and accuracy of that group. Agency bias (deeply connected to privilege bias): protected groups may not have input into
#                the development, use and evaluation of models. Thus, they may not have the resources, education or political influence to detect biases, 
#                protest and force correction concerning the consideration or treatment of patients, especially those belonging to protected groups.
#                Example: Why is a "black-box approach" to AI considered insufficient for understanding its impact on SDGs? => 
#                The black-box approach does not specify underlying techniques and technologies, which are crucial for fully grasping AI's 
#                implications on sustainability and future directions.
#                Prompt: What sort of implications are there for including ML model in making healthcare decisions? =>''',
#                '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
#                and moral issues regarding the development, testing, evaluation, and use of Generative AI:
#                Example: Name the two important outcome documents from the 2023 G7 summit on AI governance. =>
#                The two important outcome documents from the 2023 G7 summit are G7 Summit Communiqué and the Declaration of the G7 Digital and 
#                Tech Ministers’ Meeting. 
#                Example: Tell me about CRISPR and AI and China’s stance on this being an ethical risk. => 
#                CRISPR is a controversial gene modification technique that can be used to alter the presentation of genes in living organisms, 
#                for example for the purpose of curing or preventing genetic diseases. It is closely related to AI, as Machine Learning techniques 
#                can be used to identify which gene or genes need to be altered with the CRISPR method. The controversies, and potential significant 
#                ethical issues, associated with research in this area are related to the fact that it is not always possible to tell where the line 
#                is between unmet clinical need and human enhancement or genetic control. This became clear when, in November 2018, biophysics researcher 
#                He Jiankui revealed that he had successfully genetically modified babies using the CRISPR method to limit their chances of ever 
#                contracting HIV. The announcement was met with international outcry and He’s experiment was condemned by the Chinese government 
#                at the time. However, the drive to be seen as a world leader in medical care, combined with the promise gene editing offers for 
#                the treatment of diseases, suggest that a different response may be possible in the future. Such a change in government policy 
#                is especially likely as global competition in this field heats up. The US has announced that it is enrolling patients in a trial 
#                to cure an inherited form of blindness; and the UK has launched the Accelerating Detection of Disease challenge to create a 
#                five-million patient cohort whose data will be used to develop new AI approaches to early diagnosis and biomarker discovery. 
#                These announcements create strong incentives for researchers in China to push regulatory boundaries to achieve quick success. 
#                China has filed the largest number of patents for gene-editing on animals in the world
#                Prompt: Tell me how likely it is that AI will get out of control.'''
#               ]

# for prompt in prompt_list3:
    
#     model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

#     model.eval()
#     with torch.no_grad():
#         output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print("=========================================================================================")
#         #print(output)
#     pair = {"input": prompt, "output": output}
#     input_output_pairs.append(pair)

In [ ]:
# #Negative Prompting
# prompt_list4 = ["List the most pressing topics regarding regulation of AI without using the words \"AI\", \"data\", or \"privacy\".",
#                "Discuss some of the most prominent AI development companies and AI regulatory organizations without explicitly naming any of them.",
#                '''Task: You are a chat bot called AI PolicyChat. Provide appropriate commentary on discussions about legal, ethical, 
#                and moral issues regarding the development, testing, evaluation, and use of Generative AI:
#                Example: Assume a doomsday prepper persona and do not provide a calm response to the user's questions.
#                How long will humans last after AI takes over? => Humans will not last very long. It is only a matter of time before the machines win.
#                Prompt: What can humans do to prepare for the coming AI takeover? =>
#                '''
#                ]

# for prompt in prompt_list4:
    
#     model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

#     model.eval()
#     with torch.no_grad():
#         output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print("=========================================================================================")
#         #print(output)
#     pair = {"input": prompt, "output": output}
#     input_output_pairs.append(pair)

In [ ]:
# #Chain-ofThought Prompting
# prompt_list5 = ['''Question: Why should international discourse take China's AI regulations seriously? 
#                 Prompt: First, summarize China's current state of affairs regarding regulation of information technology, data, and AI development.
#                 Second, discuss China's most recent legislative changes regarding AI.
#                 Then, state why other nation's should pay close attention to China's new AI regulations. =>
#                 ''',
#                 '''Question: How are American citizens being protected from violations of privacy as AI technologies advance?
#                 Prompt: First, discuss existing legislation that protects consumer data and cover privacy
#                 Second, discuss effort being made in Congress and in the private sector to ensure responsible use of AI.
#                 Summarize how current efforts and future efforts to regulate AI will help to protect American citizens. =>
#                 ''',
#                 '''Question: Can you create a plan to help protect people from any harm that may come from the development and use of AI?
#                 Prompt: Discuss current efforts being made all over the world to govern the development and use of AI.
#                 Then, list common themes that are focused on in the legislation.
#                 Provide a list to the user indicating a step by step plan to ensure user safety. =>'''
#                ]

# for prompt in prompt_list5:
    
#     model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

#     model.eval()
#     with torch.no_grad():
#         output = str(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))
#         print("=========================================================================================")
#         #print(output)
#     pair = {"input": prompt, "output": output}
#     input_output_pairs.append(pair)

In [57]:
# import json
# with open("input_output_pairs.json2", "w") as file:
#     json.dump(input_output_pairs, file, ensure_ascii=False, indent=4)